In [169]:
import pandas as pd
import math
file = pd.read_csv('45826522P9.txt',header = None)

In [170]:
#回傳一個陣列，裡面有  除了第一列(也就是時間列)的所有value count
#呼叫時parameters放入要讀取的檔案
def countValues(file):
    file_header = list(file)
    unique_count = {}
    for i in range(1,len(file_header)):  #刻意不算時間欄位
        valueCounts = file[file_header[i]].value_counts()
        for j in range(len(valueCounts.index)):
            unique_count[valueCounts.index[j]] = valueCounts.values[j]
    return unique_count

#以下沒用到，以後要用可以參考
#unique_names = file[file_header[i]].value_counts().index.tolist() #算出個欄位unique值
#unique_count[valueCounts.indexs] = valueCounts.values
        

In [ ]:
# countValues(file)

In [171]:
countValuesList = countValues(file)

In [172]:
def activityCount(file):  #使用這個function請先確定資料是照日期排序，且第二欄位為TrafficSource
    sessionJumpCount = 0 #算session跳另一組的次數
    rowCount = 0
    for i in range(len(file.index)):
        if i==0: 
            rowCount =1 
            continue
        if file[2][i] == file[2][i-1]: #如果session連續
            rowCount += 1
        else:
            #print("換另一session")
            sessionJumpCount += 1
            rowCount +=1
            
    print("rowCount = " + str(rowCount))
    print("sessionJumpCount =" +str(sessionJumpCount))
    # print( math.log2(rowCount) + sessionJumpCount) #97
    return math.sqrt(rowCount)+sessionJumpCount

In [173]:
activityCount(file)

rowCount = 10173
sessionJumpCount =1045


1145.8612908900138

In [174]:
def purchaseActivityCount(countValuesList): #執行過countValues函數，把那個函數的return值丟進來才能算
    purchaseActivityDict = {}
    favCount = countValuesList['Fav']
    cartCount = countValuesList['Cart']
    purchaseCount = countValuesList['Purchase']
    purchaseCartConversionRate = purchaseCount/cartCount # 購物次數/cart次數
    favPurchaseConversionRate = favCount/purchaseCount # fav次數/購物次數
    #所以purchaseCartConversionRate*favCartConversionRate =  fav次數/cart次數
    favCartConversionRate = purchaseCartConversionRate*favPurchaseConversionRate # fav/Cart
    purchaseActivityDict.update({'favCount' : favCount,
                                'cartCount' : cartCount,
                                'purchaseCount' : purchaseCount,
                                'purchaseCartConversionRate' : purchaseCartConversionRate,
                                'favPurchaseConversionRate' : favPurchaseConversionRate,
                                'favCartConversionRate' : favCartConversionRate})
    return purchaseActivityDict

In [175]:
purchaseActivityCount(countValuesList)

{'favCount': 37,
 'cartCount': 142,
 'purchaseCount': 64,
 'purchaseCartConversionRate': 0.4507042253521127,
 'favPurchaseConversionRate': 0.578125,
 'favCartConversionRate': 0.2605633802816901}

In [12]:
# print(file[3][1])

ViewSalePage


In [ ]:
# for i, row in enumerate(file.values):
#     print(i)
#     print(type(row[0]))
#     print(file.values[i-1])

In [ ]:
# for i, row in enumerate(file.values):
#     #print((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds())
#     print(i)
#     if(i%3==0):
#         i+=2

In [74]:
# print(len(file.index))

10173


In [154]:
# txt = "8888 55 44"
# txt.split(" ")

['8888', '55', '44']

In [180]:
import datetime
def purchaseBeforeCount(file):
    purchase_segment_dict = {}
    a_purchase_segment_row = 0
    a_purchase_segment_product_list = {}
    a_purchase_segment_source_type = {}
    a_purchase_segment_SessionCount=0
    a_purchase_order = {}
    purchase_orders = []
    purchase_order_serial_number = 0 #訂單編號
    before_purchase_counts=[]
    i=0
    while(i+1 != len(file.index)):
        for i, row in enumerate(file.values):

            if(row[3]!='Purchase'):
                a_purchase_segment_row += 1
                try:
                    a_purchase_segment_source_type["{}".format(row[4])] = a_purchase_segment_source_type["{}".format(row[4])]+1
                except:
                    a_purchase_segment_source_type.update({"{}".format(row[4]) : 1})
                try:
                    a_purchase_segment_product_list["{}".format(row[5])] = a_purchase_segment_product_list["{}".format(row[5])]+1 #該品項或商品的count+1
                except:
                    a_purchase_segment_product_list.update({"{}".format(row[5]) : 1})
                    
                    
                
            #算purchase裡，session的跳動  => 11231為跳四次 
                if(row[2]!=file.values[i-1][2]): # i-1為上一行
                    if((pd.to_datetime(row[0]) - pd.to_datetime(file.values[i-1][0])).total_seconds()>= 1800):
                        a_purchase_segment_SessionCount += 1 
                        
            
            #如果該筆資料為purchase，把之前計算之數值放入dict,並且全部重新計算
            elif(row[3]=='Purchase'):
                
                
                before_purchase_counts.append({'segment_count' : a_purchase_segment_SessionCount,
                            'source_type' : a_purchase_segment_source_type,
                             'product_list' : a_purchase_segment_product_list
                            
                            })
#                 print(a_purchase_segment_source_type)
#                 print(a_purchase_segment_product_list)
                
                #before_purchase_list.append(before_purchase_row)
                
                
                
                a_purchase_segment_source_type = {}
                a_purchase_segment_product_list = {}
                a_purchase_segment_row = 0
                a_purchase_segment_SessionCount = 0
                
                
                #Purchase單 計算
                if(row[0] == file.values[i-1][0]): #如果時間相同
                    try:
                        a_purchase_order["{}".format(row[5]).split(" ")[0]] = a_purchase_segment_product_list["{}".format(row[5]).split(" ")[0]]+1
                    except:
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                    purchase_amount += float(row[5].split(" ")[1])
                    discount_amount += float(row[5].split(" ")[2])
                    a_purchase_order['purchase_amount'] = purchase_amount
                    a_purchase_order['discount_amount'] = discount_amount
                else:
                    if(a_purchase_order == {}):
                        a_purchase_order.update({"{}".format(row[5]).split(" ")[0] : 1})
                        a_purchase_order['purchase_amount'] = float(row[5].split(" ")[1])
                        a_purchase_order['discount_amount'] = float(row[5].split(" ")[2])
                    purchase_orders.append({'purchaseOrder{}'.format(purchase_order_serial_number) : a_purchase_order})
                    purchase_amount = float(row[5].split(" ")[1])
                    discount_amount = float(row[5].split(" ")[2])
                    purchase_order_serial_number += 1
                    a_purchase_order = {}
                    
    return before_purchase_counts,purchase_orders



#     sessionJumpCount = 0 #算session跳另一組的次數
#     rowCount = 0
#     for i in range(len(file.index)):
#         if i==0: 
#             rowCount =1 
#             continue
#         if file[2][i] == file[2][i-1]: #如果session連續
#             rowCount += 1
#         else:
#             #print("換另一session")
#             sessionJumpCount += 1
#             rowCount +=1
            
#     print("rowCount = " + str(rowCount))
#     print("sessionJumpCount =" +str(sessionJumpCount))        
            
            
    #before_purchase_list.append(每個dict)        
    #return before_purchase_list

In [181]:
a = purchaseBeforeCount(file)
a

([{'segment_count': 45,
   'source_type': {'WEB': 969, 'APP': 31},
   'product_list': {'55165': 111,
    '35222': 63,
    '4162706': 12,
    '4162097': 1,
    '4161481': 11,
    '4161230': 1,
    '4162724': 2,
    '4162111': 1,
    '4160785': 1,
    '182295': 56,
    '4142808': 1,
    '4161116': 2,
    '4162249': 2,
    '4162673': 17,
    '3853540': 4,
    '4071441': 1,
    '4162642': 3,
    '4143967': 3,
    '4106506': 1,
    '4143920': 15,
    '190037': 50,
    '4162715': 1,
    '4143319': 5,
    '4162588': 2,
    '121942': 7,
    '4133730': 5,
    '75124': 50,
    '51877': 36,
    '4162372': 2,
    '4092708': 2,
    '4010225': 1,
    '3914456': 2,
    '51880': 5,
    '4143593': 2,
    '4106259': 8,
    '231141': 3,
    '4143155': 6,
    '79812': 3,
    '4160577': 1,
    '4143793': 2,
    '3853969': 7,
    '51875': 23,
    '4092441': 6,
    '4092526': 1,
    '4010606': 1,
    '4167881': 2,
    '4143096': 1,
    '3991194': 2,
    '4010185': 2,
    '3991382': 2,
    '51876': 52,
    '4

In [178]:
len(a)

2